# Second level workflow

* Mar 24 2020 Jeesung 
* based on CNLab pipeline
* https://miykael.github.io/nipype_tutorial/notebooks/example_2ndlevel.html
* https://miykael.github.io/nipype_tutorial/notebooks/handson_analysis.html

In [3]:
import os
import json



from nipype.interfaces.matlab import MatlabCommand

# import functions
import nipype.interfaces.io as nio           # Data i/o
from nipype import Node, Workflow # Get the Node and Workflow object
from nipype.interfaces.spm import (OneSampleTTestDesign, MultipleRegressionDesign, EstimateModel,
                                   EstimateContrast, Threshold) 

from nipype.interfaces.utility import IdentityInterface
import nibabel as nb
from nipype import SelectFiles
from nipype.interfaces.io import DataSink
import nipype.pipeline.engine as pe          # pypeline engine
import nipype.interfaces.utility as util     # utility

%matplotlib inline
from nilearn import plotting
from nistats import design_matrix, reporting

import scipy.io as sio
import re
import pandas as pd
import matplotlib.pyplot as plt

from itertools import combinations, product

200427-07:50:10,748 nipype.utils INFO:
	 No new version available.


/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: FutureWarning: 

 | Starting with Nilearn 0.7.0, all Nistats functionality has been incorporated into Nilearn's stats & reporting modules.
 | Nistats package will no longer be updated or maintained.



In [2]:
# Specify which SPM to use
MatlabCommand.set_default_matlab_cmd("matlab -nodesktop -nosplash")
MatlabCommand.set_default_paths(PATH_TO_SPM_FOLDER)

NameError: name 'PATH_TO_SPM_FOLDER' is not defined

## Step1. Experiment parameters

* specify all parameters that might change between experiments

### Read json file

* set output directory

In [ ]:
# JSON_MODEL_FILE = os.path.join('/data00/projects/megameta/scripts/jupyter_megameta/second_level_models',
#                                'model_specifications','Framing',
#                                MODEL_SPEC_FILE)

# with open(JSON_MODEL_FILE) as fh:
#     model_def = json.load(fh)

### Get Contrast List

* find paths to all con images (=input)

In [14]:
ROOT_DIR = '/data00/projects/megameta'

In [186]:
def make_contrast_list(model_path, cname, exclude_subjects, sample_perc=100):
    #EDITED BY CHRISTIN to get randomly sample a given percentage of subjects for second-level model

    import json
    import random
    import os
    import scipy.io as sio
    import pandas as pd
    
    ROOT_DIR = '/data00/projects/megameta'
    
    
    def get_mreg(model_path, DEBUG=True):
        import json
        import os
        import pandas as pd

        ROOT_DIR = '/data00/projects/megameta'


        with open(model_path) as fh:
            model_def = json.load(fh)

        if not model_def:
            return None

        mreg_file='{}.csv'.format(model_def['Regressors']['Name'])

        mreg_cols = model_def['Regressors']['Columns']

        project_phenotype_file = [os.path.join(ROOT_DIR,project['Name'], 'phenotype', mreg_file) for project in model_def['Projects']]

        data=[]
        for p in project_phenotype_file:
            if not os.path.exists(p):
                print('ERROR cannot find', p)
            else:
#                 df=pd.read_csv(p, sep='\t')
                df=pd.read_csv(p)
                # check to see if the participant_id column has compliant BIDS subject ids with sub- format
                df.loc[-df['participant_id'].str.startswith('sub-'), 'participant_id'] = 'sub-'+df['participant_id']


                # drop rows with NAs in regressor columns
                df = df[df[mreg_cols].notnull().all(axis=1)]

                data.append(df)

        return pd.concat(data), mreg_cols
    
    
    def process_project(project_name, model_def,exclude_subjects=exclude_subjects, scan_all_subjs=False, DEBUG=False):

        project_spec = [pspec for pspec in model_def['Projects'] if pspec['Name']==project_name]

        if not project_spec:
            print('Cannot find specification for project: ', project_name)
            return None

        model_name = project_spec[0]['Model']
        cmap = project_spec[0]['ContrastMap']


        model_dir = os.path.join(ROOT_DIR, project_name, 
                                 "derivatives", "nipype",
                                 "model_{}".format(model_name)
                                )

        if not os.path.exists(model_dir):
            print('Cannot find first level model directory:', model_dir)
            return None

        subjs_with_models = [s for s in os.listdir(model_dir) if s.startswith('sub-')]
        #exclude_people
        subjs_with_models=[s for s in subjs_with_models if s not in exclude_subjects]
        
        print('excluded subject list: ',exclude_subjects, ' (N=', len(exclude_subjects), ')')
        print('included subject list: ',subjs_with_models,' (N=', len(subjs_with_models),')')

        
        #Get a random sample of participants (based on a percentage)
        sample_size=(sample_perc/100)*len(subjs_with_models)
        subj_list=random.sample(subjs_with_models,int(sample_size))
        
        print('Project: {}, Sampling {} of {} participants with a model'.format(project_name, int(sample_size), len(subj_list)))
        
        if DEBUG:
            print("Found {} first level subject models\n".format(len(subjs_with_models)))


        contrast_lists = { cname: [] for cname in cmap}


        model_contrasts=None
        for sidx,subj in enumerate(subj_list):

            if DEBUG:
                print('Processing',subj, '-',end='')

            first_level_dir = os.path.join(model_dir, subj, 'medium', 'fwhm_8')

            if scan_all_subjs or sidx==0:
                spm_mat_file = os.path.join(first_level_dir, 'SPM.mat')

                SPM = sio.loadmat(spm_mat_file, squeeze_me=True, struct_as_record=False)['SPM']

                model_contrasts = SPM.xCon

            if DEBUG:
                print(' found {} contrasts'.format(len(model_contrasts)))

            con_map = {con.name: 'con_{:0>4}.nii'.format(cidx) for cidx,con in enumerate(model_contrasts,1) }


            if DEBUG:
                print('\tContrasts are:', con_map)

            for model_con, proj_con in cmap.items():

                path_to_con = os.path.join(first_level_dir, con_map[proj_con])

                if os.path.exists(path_to_con):
                    contrast_lists[model_con].append(path_to_con)

        return contrast_lists, subjs_with_models

    with open(model_path) as fh:
        model_def = json.load(fh)
        if model_def.get('Regressors',False):
            mreg_df, mreg_cols=get_mreg(model_path)

        

        
    conlist=[]
    subjs_with_models=[]
    for p in model_def['Projects']:
        cons, subjs=process_project(p['Name'], model_def, exclude_subjects)
        conlist.extend(cons[cname])
    
    
    con_df = pd.DataFrame(conlist, columns=['conpath'])
    con_df['participant_id'] = con_df['conpath'].apply(lambda cp: cp.split('/')[8])
    
    final_df=pd.merge(con_df,mreg_df,on='participant_id')

        
    mregs=[]
    for k,v in final_df[mreg_cols].to_dict(orient='list').items():
        mregs.append({'name': k, 'vector': v, 'centering': 5})   # value of 5 for centering is iCC = 5 (no centuring in the spm_factorial model)

    
    con_list = final_df['conpath'].values.tolist()


    print('Participants with both first level models and behavior data N=',len(mregs[0]['vector']))
    return con_list, mregs

In [216]:
l2_getcontrasts = pe.Node(util.Function(input_names=['model_path','cname','exclude_subjects'],
                                     output_names=['contrasts','covariates'],
                                    function=make_contrast_list), name='makecontrasts')

l2_getcontrasts.inputs.model_path=JSON_MODEL_FILE
l2_getcontrasts.inputs.cname=contrast_name
l2_getcontrasts.inputs.exclude_subjects=exclude_subjects

### Load explicit mask

In [26]:
# mask_path='/data00/tools/spm8/apriori/brainmask_th25.nii'
# mask = nb.load(mask_path)
# # mask.orthoview()
# # plotting.view_img(mask1) # interactive

## Step 2. Create Nodes

### 2nd-Level Design 

* one-sample t-test design

* This step depends on your study design and the tests you want to perform. If you're using SPM to do the group analysis, you have the liberty to choose between a factorial design, a multiple regression design, one-sample T-Test design, a paired T-Test design or a two-sample T-Test design.



In [214]:
# # Infosource - a function free node to iterate over the list of subject names
# l2_infosource = pe.Node(util.IdentityInterface(fields=['contrast_id']),
#                   name="infosource")

# smoothing_kernels = [ 8 ]
# resolutions = ['medium']

# resolution_and_kernel_list = product(resolutions, smoothing_kernels)


# l2_infosource.iterables = [('contrast_id', contrast_name), 
#                            ('resolution_and_smoothing', resolution_and_kernel_list)
#                         ]

In [27]:
# Initiate the OneSampleTTestDesign node here
onesamplettestdes = Node(OneSampleTTestDesign(), name="onesampttestdes")

# specify the binary mask as an explicit_mask_file for the one-sample T-test node.
onesamplettestdes.inputs.explicit_mask_file=mask_path
onesamplettestdes.inputs.threshold_mask_none=True

In [31]:
# Multiple Regression Design - creates mreg Design
mregdesign = Node(MultipleRegressionDesign(),name="mregdesign")
mregdesign.inputs.threshold_mask_none=True

# mregs=make_contrast_list(JSON_MODEL_FILE, contrast_name,exclude_subjects)[1]
# # # Add covariates to control for behavior change
# mregdesign.inputs.covariates=l2_getcontrasts.outputs.covariates

# print('Participants with both first level models and behavior data N=',len(mregs[0]['vector']))

NameError: name 'mregs' is not defined

In [5]:
# Initiate the EstimateModel and the EstimateContrast node here

level2estimate = Node(EstimateModel(estimation_method={'Classical': 1}),
                      name="level2estimate")

level2conestimate = Node(EstimateContrast(group_contrast=True),
                         name="level2conestimate")


# cont1 = ['Change', 'T', ['change','baseline','mean'], [1,0,0]]
# cont2 = ['Baseline', 'T', ['change','baseline','mean'], [0,1,0]]
# cont3 = ['Group', 'T', ['change','baseline','mean'], [0,0,1]]

# cont1 = ['Change', 'T', ['change','baseline','mean'], [1,0,0]]
# cont2 = ['Baseline', 'T', ['change','baseline','mean'], [0,1,0]]
# cont3 = ['Group', 'T', ['change','baseline','mean'], [0,0,1]]

level2conestimate.inputs.contrasts = contrast_list

### Thresholding of output contrast

In [198]:
# level2thresh = Node(Threshold(contrast_index=1,
#                               use_topo_fdr=True,
#                               use_fwe_correction=False,
#                               extent_threshold=0,
#                               height_threshold_type='p-value',
#                               extent_fdr_p_threshold=0.05),
#                     name="level2thresh")

### Specify input stream (SPM12)

#### Datainput with SelectFiles and iterables

* data in use: 1st level contrasts of all subjects, separated by contrast number
* " * " = tell SelectFiles that it can grab available subjects and any contrast with a specific contrast id 

* replace this step with 'make_contrast_list' function

### Data output with DataSink

* specify a Datasink folder to only keep those files that we want to keep.

In [38]:
# Initiate DataSink node, create output folder for important outputs
# new_output_dir=os.path.join(output_dir,contrast_name)
datasink = Node(DataSink(base_directory=output_dir,
                         container=output_dir),
                name="datasink")

## Use the following substitutions for the DataSink output
substitutions = [('_contrast_id_','')] # change this 
datasink.inputs.substitutions = substitutions

Now the next step is to specify all the output that we want to keep in our output folder output. Probably best to keep are the:

* the SPM.mat file and the spmT images from the EstimateContrast node
* the thresholded spmT images from the Threshold node

## Step 3. Specify Workflow (SPM12)
* Create a workflow and connect the interface nodes and the input/output stream to each other.



In [8]:
# each node will be constrcuted within the workflow named 'l2analysis'
l2analysis=Workflow(name='l2analysis', base_dir=working_dir)

NameError: name 'working_dir' is not defined

In [13]:
# Connect up the 2nd-level analysis components
l2analysis.connect([(l2_getcontrasts,  mregdesign, [('contrasts', 'in_files'),
                                                     ('covariates','covariates')]),
                    (mregdesign, level2estimate, [('spm_mat_file',
                                                          'spm_mat_file')]),
                    (level2estimate, level2conestimate, [('spm_mat_file',
                                                          'spm_mat_file'),
                                                         ('beta_images',
                                                          'beta_images'),
                                                         ('residual_image',
                                                          'residual_image')]),
                    (level2conestimate, datasink, [('spm_mat_file',
                                                    'contrasts.@spm_mat'),
                                                   ('spmT_images',
                                                    'contrasts.@T'),
                                                   ('con_images',
                                                    'contrasts.@con')])
                    ])

NameError: name 'l2analysis' is not defined

In [34]:
# # Connect up the 2nd-level analysis components
# l2analysis.connect([(l2_getcontrasts,  mregdesign, [('contrasts', 'in_files')]),
# #                                                      ('covariates', 'covariates')]),
#                     (mregdesign, level2estimate, [('spm_mat_file',
#                                                           'spm_mat_file')] ),
#                     (level2estimate, level2conestimate, [('spm_mat_file',
#                                                           'spm_mat_file'),
#                                                          ('beta_images',
#                                                           'beta_images'),
#                                                          ('residual_image',
#                                                           'residual_image')]),
#                     (level2conestimate, level2thresh, [('spm_mat_file',
#                                                         'spm_mat_file'),
#                                                        ('spmT_images',
#                                                         'stat_image')]),
#                     (level2conestimate, datasink, [('spm_mat_file',
#                                                     'contrasts.@spm_mat'),
#                                                    ('spmT_images',
#                                                     'contrasts.@T'),
#                                                    ('con_images',
#                                                     'contrasts.@con')]),
#                     (level2thresh, datasink, [('thresholded_map',
#                                                'contrasts.@threshold')])
#                     ])

NameError: name 'l2analysis' is not defined